In [25]:
from collections import defaultdict

import numpy as np
import pandas as pd
from bed_reader import open_bed, sample_file

from src.model.porpoise import Model

In [2]:
with open('/Users/arish/Workspace/experiments/rna_modification/dataset/raw/human.hg38.Pseudo.result.col29.bed') as file:
    lines1 = file.readlines()

with open('/Users/arish/Workspace/experiments/rna_modification/dataset/raw/RMBase_hg19_all_PseudoU_site.txt') as file:
    lines2 = file.readlines()

In [50]:
sequences = []

for line in lines1:
    line = line.strip()
    items = line.split('\t')
    sequences.append((int(items[1]), int(items[2]), items[3], items[18].replace('T', 'U')))

In [51]:
sequences

[(629454,
  629455,
  'Pseudo_site_1',
  'AAAUAUGUCUGAUAAAAGAGUUACUUUGAUAGAGUAAAUAA'),
 (629455,
  629456,
  'Pseudo_site_2',
  'AAUAUGUCUGAUAAAAGAGUUACUUUGAUAGAGUAAAUAAU'),
 (629594,
  629595,
  'Pseudo_site_3',
  'AAGGUCAGCUAAAUAAGCUAUCGGGCCCAUACCCCGAAAAU'),
 (629616,
  629617,
  'Pseudo_site_4',
  'GGGCCCAUACCCCGAAAAUGUUGGUUAUAUCCUUCCCGUAC'),
 (632711,
  632712,
  'Pseudo_site_5',
  'AUAUUAGAAAAACCAUUUCAUAACUUUGUCAAAGUUAAAUU'),
 (633484,
  633485,
  'Pseudo_site_6',
  'ACUGUAAAGCUAACUUAGCAUUAACCUUUUAAGUUAAAGAU'),
 (633485,
  633486,
  'Pseudo_site_7',
  'CUGUAAAGCUAACUUAGCAUUAACCUUUUAAGUUAAAGAUU'),
 (873706,
  873707,
  'Pseudo_site_8',
  'CUGAACCACCUGGUGUCUGCUACCAUGAGUGGGGUCACCAC'),
 (1045799,
  1045800,
  'Pseudo_site_9',
  'AUGACGGGGCUGUGCUCGUGUAAGCCCGGGGUGGCUGGACC'),
 (1051520,
  1051521,
  'Pseudo_site_10',
  'CAGGUGGACGUCACGUCCUUUGCAGGUCACCCCUGCACCCG'),
 (1254956,
  1254957,
  'Pseudo_site_11',
  'GGCCUUGUGUGCUGGAUUUGUAGCUUAUCUUCCGUGUUGUC'),
 (1257012,
  1257013,
  'Pseudo_sit

In [52]:
sorted_sequences = sorted(sequences, key=lambda x: x[0])

In [59]:
for i in range(1, len(sorted_sequences)):
    # CGGCAGCACAUAUACUAAAAUUGGAACGAUACAGAGAAAAC
    # CGGCAGCACAUAUACUAAAAUUGGAACGAUACAGAGAAAAC
    print(sorted_sequences[i - 1], sorted_sequences[i])

(30, 31, 'Pseudo_site_5658', 'CGGCAGCACAUAUACUAAAAUUGGAACGAUACAGAGAAAAC') (41, 42, 'Pseudo_site_5657', 'CGGCAGCACAUAUACUAAAAUUGGAACGAUACAGAGAAAAC')
(41, 42, 'Pseudo_site_5657', 'CGGCAGCACAUAUACUAAAAUUGGAACGAUACAGAGAAAAC') (41, 42, 'Pseudo_site_5659', 'CGGCAGCACAUAUACUAAAAUUGGAACGAUACAGAGAAAAC')
(41, 42, 'Pseudo_site_5659', 'CGGCAGCACAUAUACUAAAAUUGGAACGAUACAGAGAAAAC') (41, 42, 'Pseudo_site_5669', 'CGGCAGCACAUAUACUAAAAUUGGAACGAUACAGAGAAAAC')
(41, 42, 'Pseudo_site_5669', 'CGGCAGCACAUAUACUAAAAUUGGAACGAUACAGAGAAAAC') (41, 42, 'Pseudo_site_5678', 'CGGCAGCACAUAUACUAAAAUUGGAACGAUACAGAGAAAAC')
(41, 42, 'Pseudo_site_5678', 'CGGCAGCACAUAUACUAAAAUUGGAACGAUACAGAGAAAAC') (41, 42, 'Pseudo_site_5681', 'CGGCAGCACAUAUACUAAAAUUGGAACGAUACAGAGAAAAC')
(41, 42, 'Pseudo_site_5681', 'CGGCAGCACAUAUACUAAAAUUGGAACGAUACAGAGAAAAC') (42, 43, 'Pseudo_site_5672', 'CGGCAGCACAUAUACUAAAAUUGGAACGAUACAGAGAAAAC')
(42, 43, 'Pseudo_site_5672', 'CGGCAGCACAUAUACUAAAAUUGGAACGAUACAGAGAAAAC') (224, 225, 'Pseudo_site_5654', 'CGGCAG

In [11]:
sequences1 = []
sequences2 = []

for line in lines1:
    line = line.strip()
    items = line.split('\t')
    sequences1.append(items[18].replace('T', 'U'))


for line in lines2:
    line = line.strip()
    items = line.split('\t')
    sequences2.append(items[14].replace('T', 'U'))

In [66]:
not_found_sequences1 = []
not_found_sequences2 = []

for seq1 in sequences1:
    if seq1 not in sequences2:
        not_found_sequences1.append(seq1)

for seq2 in sequences2:
    if seq2 not in sequences1:
        not_found_sequences2.append(seq2)

print(len(not_found_sequences1))
print(len(not_found_sequences2))

989
130


In [65]:
unique_21_sequences = set()

for seq1 in sequences1:
    unique_21_sequences.add(seq1[5:-5])

for seq2 in sequences2:
    unique_21_sequences.add(seq2[5:-5])

print(len(unique_21_sequences))

3521


In [67]:
from src.dataset import load_benchmark_dataset, Species, Modification
from src.features.encodings import multiple, binary
# load_benchmark_dataset(Species.human, Modification.psi).targets

In [68]:
unique_seq_dataframe = pd.DataFrame(unique_21_sequences, columns=['sequence'])

In [69]:
encoder = binary.Encoder()
transformed_sequences = encoder.fit_transform(unique_seq_dataframe, y=pd.Series([1] * len(unique_seq_dataframe), name=1))

In [70]:
from sklearn.svm import OneClassSVM

In [71]:
transformed_sequences

,binary_0,binary_1,binary_2,binary_3,binary_4,binary_5,binary_6,binary_7,binary_8,binary_9,...,binary_114,binary_115,binary_116,binary_117,binary_118,binary_119,binary_120,binary_121,binary_122,binary_123
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3516,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3517,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3518,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3519,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [ ]:
from sklearn.ensemble import IsolationForest

In [72]:
model = OneClassSVM().fit(transformed_sequences)
predictions = model.predict(transformed_sequences)

In [73]:
np.sum(predictions == 1)

1735

In [74]:
1735 / len(predictions)

0.49275773927861405